#### 一、参数的更新

##### 1.1、SGD的故事你讲一下

- 一个探险家下山：一不看地图 二把眼蒙上，就靠”脚感“决定下一步走哪，心里想着只要重复这一策略够久就可以到达”至深之地“。

##### 1.2、SGD中Loss会偶尔上升正常吗？
- 你看过李宏毅的我的世界动画，梯度的方向是各偏微分方向的矢量和，因此指向上是正常的。

##### 1.3、SGD低效的根本原因是什么？
- $\boldsymbol{W} \leftarrow \boldsymbol{W}-\eta \frac{\partial L}{\partial \boldsymbol{W}}$
- SGD低效的根本原因：梯度的方向并没有指向最小值的方向！
![](./attachements/SGD低效的例子.png)
![](./attachements/SGD例子梯度下降图.png)
![](./attachements/SGD在函数非均向时候的搜索路径.png)

##### 1.4、Momentum动量法是怎么指导参数每次的更新量不只取决于当前的梯度？

$$
\begin{array}{c}
\boldsymbol{v} \leftarrow \alpha \boldsymbol{v}-\eta \frac{\partial L}{\partial \boldsymbol{W}} \\
\boldsymbol{W} \leftarrow \boldsymbol{W}+\boldsymbol{v}
\end{array}
$$

> 你怎么理解速度v是历史梯度的积累？
- 你看公式不难理解，但是看代码最好理解，那个W是＋的$\alpha v$,上次速度越大这次就越大！此为"惯性"。直观的理解就是如果连续几次更新的方向一致（更快的穿过平缓的区域），那么速度就会不断叠加，使得v越来越大，加速收敛！
![](./attachements/Momentum的最优化路径.png)

##### 1.5、AdaGrad自适应学习率算法是怎么调整学习率的？

$$
\begin{array}{c}
\boldsymbol{h} \leftarrow \boldsymbol{h} + \frac{\partial L}{\partial \boldsymbol{W}} \odot \frac{\partial L}{\partial \boldsymbol{W}} \\
\boldsymbol{W} \leftarrow \boldsymbol{W} - \eta \frac{1}{\sqrt{\boldsymbol{h}}} \frac{\partial L}{\partial \boldsymbol{W}}
\end{array}
$$

解释：
- 因为 $\boldsymbol{h}$ 累积了每个参数的梯度平方，从而为每个参数计算一个不同的步长。
- **梯度大则参数的学习步伐小，梯度小则参数的学习步伐大**，这是由公式中 $\eta / \sqrt{\boldsymbol{h}}$ 实现的。


##### 1.6、RMSProp均方根误差传播算法是怎么实现逐渐遗忘过去的梯度的？
> 为什么要逐渐遗忘过去的梯度，因为你看AdaGrad最后在某些历史梯度上越来越小的更新到无穷小是没有意义的

1. **累积梯度平方的指数加权移动平均：**
   $$
   \boldsymbol{s} \leftarrow \beta \boldsymbol{s} + (1 - \beta) \frac{\partial L}{\partial \mathbf{W}} \odot \frac{\partial L}{\partial \mathbf{W}}
   $$
2. **权重更新公式：**
   $$
   \mathbf{W} \leftarrow \mathbf{W} - \eta \frac{1}{\sqrt{\boldsymbol{s} + \epsilon}} \frac{\partial L}{\partial \mathbf{W}}
   $$
**核心总结**：
- **指数加权移动平均**就是通过 $\beta \boldsymbol{s} + (1 - \beta) \text{当前梯度平方}$ 来实现的。
- $\beta$ 越大，越多保留过去信息，逐渐遗忘历史的影响。

这样，RMSProp 可以适应梯度变化，而不会像 AdaGrad 那样累积所有历史梯度。

##### 1.7、Adam是怎么融合AdaGrad + Momentum的呢？


1. **动量部分（Momentum）**：
   - 动量更新公式：$ \mathbf{m}_t = \beta_1 \mathbf{m}_{t-1} + (1 - \beta_1) \frac{\partial L}{\partial \mathbf{W}} $
   - **作用**：累积梯度的方向，使更新更平滑，具有加速效果。

2. **自适应学习率部分（类似 AdaGrad）**：
   - 累积梯度平方公式：$ \mathbf{v}_t = \beta_2 \mathbf{v}_{t-1} + (1 - \beta_2) \left( \frac{\partial L}{\partial \mathbf{W}} \odot \frac{\partial L}{\partial \mathbf{W}} \right) $
   - **作用**：累积梯度平方，动态调整学习步长，防止学习率过大或过小。

3. **最终更新**：
   - 权重更新公式：$ \mathbf{W} \leftarrow \mathbf{W} - \eta \frac{\hat{\mathbf{m}}_t}{\sqrt{\hat{\mathbf{v}}_t} + \epsilon} $
   - **结合**：动量部分（平滑方向） + 自适应学习率（根据梯度动态调整步长）。

4. **总结**：
- Adam = **Momentum**（方向加速） + **AdaGrad**（学习率自适应）。

##### 1.8、怎么选择最好的优化器呢？
> 没有最好的，根据自己的喜好多多尝试。



#### 二、权重的初始值设置

> 权重初始值的重要性容易被忽视，而任何事情的开始（初始值）总是关键的

##### 2.1、可以将神经网络中权重的初始值设置为0吗？
- 你画个计算图就能看出来：权重初始为0就意味着梯度传播无法进行，网络完全学不到任何东西

##### 2.2、可以将神经网络中权重的初始值设置为相同的一个值吗？
- 权重初始为相同的非零值导致网络失去差异性，各个神经元在整个训练过程中表现一致，最终也无法有效地学习到不同特征。

##### 2.3、怎么设计一个实验来研究隐藏层的激活值的分布？
> 就看各层的激活值呗，自变量是不同的W值（标准差为1的高斯分布、为0.01的高斯分布。。。）
![](./attachements/使用标准差为1的高斯分布作为权重初始值时的各层激活值的分布.png)
- 梯度消失现象：反向传播中梯度的值太小了，跟没有一样！（这里使用的 sigmoid
函数是 S 型函数，随着输出不断地靠近 0（或者靠近 1），它的导数的值逐渐接
近 0。因此，偏向 0 和 1 的数据分布会造成反向传播中梯度的值不断变小，最
后消失。这个问题称为梯度消失（gradient vanishing）。）
![](./attachements/使用标准差为0.01的高斯分布作为权重初始值时的各层激活值分布.png)
- 表现力受限现象：10个相同的员工干着相同的活儿。因为如果有多个神经元都输出几乎相同
的值，那它们就没有存在的意义了。比如，如果 100 个神经元都输出几乎相
同的值，那么也可以由 1 个神经元来表达基本相同的事情。

##### 2.4、Xavier初始值是怎么设定的初始值使得各层的激活值呈现出具有相同广度的分布？
![](./attachements/Xavier初始值.png)

1. **均匀分布初始化**：
   $$
   W \sim \mathcal{U}\left( -\frac{\sqrt{6}}{\sqrt{n_{\text{in}} + n_{\text{out}}}}, \frac{\sqrt{6}}{\sqrt{n_{\text{in}} + n_{\text{out}}}} \right)
   $$
   - $W$：权重矩阵
   - $n_{\text{in}}$：输入节点数量
   - $n_{\text{out}}$：输出节点数量
   - $\mathcal{U}(a, b)$：均匀分布，范围从 $-a$ 到 $b$

2. **正态分布初始化**：
   $$
   W \sim \mathcal{N}\left( 0, \frac{1}{\sqrt{n_{\text{in}}}} \right)
   $$
   - $W$：权重矩阵
   - $\mathcal{N}(\mu, \sigma^2)$：正态分布，均值为 $0$，方差为 $\frac{1}{n_{\text{in}}}$

这两种方式确保激活值的方差稳定，避免梯度消失或爆炸，保持训练的稳定性。

##### 2.5、为什么用作激活函数的函数最好具有关于原点对称的性质？

##### 2.6、为什么ReLU还要专门搞一个专用的He初始值？

> 因为Xavier 初始值是以激活函数是线性函数为前提而推导出来的。ReLU 的激活特性会导致很多神经元输出为零，这使得激活值的方差容易减小，导致梯度消失。He 初始化通过增大初始权重的方差（为 $\frac{2}{n_{\text{in}}}$），使得激活值的分布在经过 ReLU 激活后保持更合理的广度。


1. **均匀分布初始化**：
   $$
   W \sim \mathcal{U}\left( -\frac{\sqrt{6}}{\sqrt{n_{\text{in}}}}, \frac{\sqrt{6}}{\sqrt{n_{\text{in}}}} \right)
   $$
   - $W$：权重矩阵
   - $n_{\text{in}}$：输入节点数量
   - $\mathcal{U}(a, b)$：均匀分布，范围从 $-a$ 到 $b$

2. **正态分布初始化**：
   $$
   W \sim \mathcal{N}\left( 0, \frac{2}{n_{\text{in}}} \right)
   $$
   - $W$：权重矩阵
   - $\mathcal{N}(\mu, \sigma^2)$：正态分布，均值为 $0$，方差为 $\frac{2}{n_{\text{in}}}$

3. 核心目标
- **He 初始化**专为 **ReLU** 激活函数设计，确保激活值的方差适应 ReLU 的性质，避免深度网络中的梯度消失或爆炸。


#### 三、Batch Normalization

##### 3.1、Batch Norm是个搞什么的？
• 可以使学习快速进行（可以增大学习率）。
• 不那么依赖初始值（对于初始值不用那么神经质）。
• 抑制过拟合（降低 Dropout 等的必要性）。

##### 3.2、Batch Norm是通过什么方法做到对权重初始值脱敏的？

![](./attachements/Batch%20Normalization.png)
- **标准化激活值**：对每一层的激活值进行零均值和单位方差的标准化。
- **效果**：无论权重初始值如何，激活值会被规范化成一个稳定的分布，这样就使得网络对初始权重的选择不再那么敏感。


#### 四、正则化

##### 4.1、发生过拟合的原因是什么？
• 模型拥有大量参数、表现力强。
• 训练数据少。
> 比如从MNIST 数据集原本的 60000 个训练数据中只选定 300 个，并且，为了增加网络的复杂度，使用 7 层网络（每层有 100 个神经元，激活函数为 ReLU）。

##### 4.2、抑制过拟合的思路是什么？
> 很多过拟合原本就是因为权重参数取值过大才发生的。
• 权值衰减: 惩罚大权重
• Dropout: 随机删除神经元


##### 4.3、权值衰减家族有哪些成员？
• L1: 惩罚项为**权重绝对值之和**，导致稀疏的权重（很多权重为零）$\lambda \sum |w|$
• L2:惩罚项为**权重平方和**，鼓励小而均匀的权重，较为平滑。$\lambda \sum w^2$
• Elastic Net:结合 L1 和 L2，既得到稀疏性（L1），又有平滑性（L2）$\lambda_1 \sum |w| + \lambda_2 \sum w^2$

##### 4.4、Dropout是什么？为什么能模拟集成学习的效果？
![](./attachements/Dropout.png)

- 所谓集成学习，就是让多个模型单独进行学习，推理时再取多个模型的输出的平均值。用神经网络的语境来说，比如，准备 5 个结构相同（或者类似）的网络，分别进行学习，测试时，以这 5 个网络的输出的平均值作为答案。
- 可以将 Dropout理解为，通过在学习过程中随机删除神经元，从而每一次都让不同的模型进行学习。也就是说，可以理解成，Dropout 将集成学习的效果（模拟地）通过一个网络实现了。


#### 五、怎么调超参数

##### 5.1、神经网络的超参数有哪些？
- 各层的神经元数量
- batch大小
- 学习率
- 权值衰减策略

##### 5.2、为什么训练集还要分一杯羹给验证集？用测试集不行？
- 因为如果使用测试数据调整超参数，超参数的值会对测试数据发生过拟合。
- 训练数据：用于参数（权重、偏置）的学习
- 验证数据：超参数的性能评估
- 测试数据：确认泛化能力。要在最后使用（比较理想的是只用一次）

##### 5.3、分割训练集为训练集+测试集时为什么要先shuffle打乱？
- 因为数据集的数据可能存在偏向（比如，数据从“0”到“10”按顺序排列等）。

##### 5.4、超参数调优的实践步骤是什么？
> 逐渐缩小“好值”存在的范围是搜索超参数的一个有效方法

##### 5.5、设计一个实验去调超参的思路与步骤！
- 步骤1：设定超参数的范围
- 步骤2：从设定的超参数范围中随机采样
- 步骤3：使用步骤2中采样到的超参数的值进行学习，通过验证数据评估识别精度。（epoch小一点不然费时）
- 步骤4：重复步骤2、3（100次等），根据他们的识别精度结果，缩小超参数的范围。